# nuScenes devkit tutorial

Welcome to the nuScenes tutorial. This demo assumes the database itself is available at `./data/`, and loads a mini version of the full dataset.

## A Gentle Introduction to nuScenes

In this part of the tutorial, let us go through a top-down introduction of our database. Our dataset comprises of elemental building blocks that are the following:

1. `log` - Log information from which the data was extracted.
2. `scene` - 20 second snippet of a car's journey.
3. `sample` - An annotated snapshot of a scene at a particular timestamp.
4. `sample_data` - Data collected from a particular sensor.
5. `ego_pose` - Ego vehicle poses at a particular timestamp.
6. `sensor` - A specific sensor type.
7. `calibrated sensor` - Definition of a particular sensor as calibrated on a particular vehicle.
8. `instance` - Enumeration of all object instance we observed.
9. `category` - Taxonomy of object categories (e.g. vehicle, human). 
10. `attribute` - Property of an instance that can change while the category remains the same.
11. `visibility` - Fraction of pixels visible in all the images collected from 6 different cameras.
12. `sample_annotation` - An annotated instance of an object within our interest.
13. `map` - Map data that is stored as binary semantic masks from a top-down view.

The database schema is visualized below. For more information see the [nuScenes schema](https://github.com/nutonomy/nuscenes-devkit/blob/master/docs/schema_nuscenes.md) page.
![](https://www.nuscenes.org/public/images/nuscenes-schema.svg)

## Initialization

In [1]:
import sys

sys.path.append('./build')

from _nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='./data/', verbose=True)

Loading NuScenes dataset for version v1.0-mini
Loaded 23 category.
Loaded 8 attribute.
Loaded 4 visibility.
Loaded 911 instance.
Loaded 12 sensor.
Loaded 120 calibrated_sensor.
Loaded 31206 ego_pose.
Loaded 8 log.
Loaded 10 scene.
Loaded 404 sample.
Loaded 31206 sample_data.
Loaded 18538 sample_annotation.
Loaded 4 map.
Loaded data in 589 ms
Built reverse index in 29 ms


## A look at the dataset

### 1. `scene`

nuScenes is a large scale database that features annotated samples across ***1000 scenes*** of approximately 20 seconds each. Let's take a look at the scenes that we have in the loaded database.

In [2]:
nusc.scenes

Let's look at a scene metadata

In [3]:
my_scene = nusc.scenes[0]
my_scene

### 2. `sample`

In scenes, we annotate our data every half a second (2 Hz).

We define `sample` as an ***annotated keyframe of a scene at a given timestamp***. A keyframe is a frame where the time-stamps of data from all the sensors should be very close to the time-stamp of the sample it points to.

Now, let us look at the first annotated sample in this scene.

In [4]:
first_sample_token = my_scene.first_sample_token
first_sample_token

'ca9a282c9e77460f8360f564131a8af5'

Let's examine its metadata

In [5]:
my_sample = nusc.sample(first_sample_token)
my_sample

A useful method is  `list_sample()` which lists all related `sample_data` keyframes and `sample_annotation` associated with a `sample` which we will discuss in detail in the subsequent parts.

In [6]:
print(len(my_sample.annotations))
print(len(my_sample.datas))

69
12


### 3. `sample_data`

The nuScenes dataset contains data that is collected from a full sensor suite. Hence, for each snapshot of a scene, we provide references to a family of data that is collected from these sensors. 

We provide a `data` key to access these:

In [7]:
my_sample.datas

### 4. `sample_annotation`

`sample_annotation` refers to any ***bounding box defining the position of an object seen in a sample***. All location data is given with respect to the global coordinate system. Let's examine an example from our `sample` above.

In [8]:
my_annotation_token = my_sample.annotations[18].token
my_annotation_metadata =  nusc.annotation(my_annotation_token)
my_annotation_metadata

### 5. `instance`

Object instance are instances that need to be detected or tracked by an AV (e.g a particular vehicle, pedestrian). Let us examine an instance metadata

In [9]:
my_instance = nusc.instances[599]
my_instance

We generally track an instance across different frames in a particular scene. However, we do not track them across different scenes. In this example, we have 16 annotated samples for this instance across a particular scene.

In [10]:
instance_token = my_instance.token

### 6. `category`

A `category` is the object assignment of an annotation.  Let's look at the category table we have in our database. The table contains the taxonomy of different object categories and also list the subcategories (delineated by a period). 

In [11]:
nusc.categories

A category record contains the name and the description of that particular category.

In [12]:
nusc.categories[9]

Refer to `instructions_nuscenes.md` for the definitions of the different categories.

### 7. `attribute`

An `attribute` is a property of an instance that may change throughout different parts of a scene while the category remains the same. Here we list the provided attributes and the number of annotations associated with a particular attribute.

In [13]:
nusc.attributes

Let's take a look at an example how an attribute may change over one scene

In [14]:
my_instance = nusc.instances[27]
first_token = my_instance.first_annotation_token
last_token = my_instance.last_annotation_token
nbr_samples = my_instance.nbr_annotations
current_token = first_token

found_change = False
last_attr = None
for ann in my_instance.annotations:
    cur_attr_token = ann.attribute_tokens[0]
    cur_attr = nusc.attribute(cur_attr_token).name
    if cur_attr != last_attr:
        print("Changed from `{}` to `{}` at timestamp {} out of {} annotated timestamps".format(last_attr, cur_attr, ann.sample.timestamp, nbr_samples))
        found_change = True
    last_attr = cur_attr


Changed from `None` to `pedestrian.moving` at timestamp 1532402927647951 out of 39 annotated timestamps
Changed from `pedestrian.moving` to `pedestrian.standing` at timestamp 1532402938197897 out of 39 annotated timestamps


### 8. `visibility`

`visibility` is defined as the fraction of pixels of a particular annotation that are visible over the 6 camera feeds, grouped into 4 bins.

In [15]:
nusc.visibilities

Let's look at an example `sample_annotation` with 80-100% visibility

In [16]:
anntoken = 'a7d0722bce164f88adf03ada491ea0ba'
visibility = nusc.annotation(anntoken).visibility_token

print(f"Visibility: {visibility}")

Visibility: 4


Let's look at an example `sample_annotation` with 0-40% visibility

In [17]:
anntoken = '9f450bf6b7454551bbbc9a4c6e74ef2e'
visibility_token = nusc.annotation(anntoken).visibility_token
visibility = nusc.visibility(visibility_token).token

print(f"Visibility: {visibility}")

Visibility: 1


### 9. `sensor`

The nuScenes dataset consists of data collected from our full sensor suite which consists of:
- 1 x LIDAR, 
- 5 x RADAR, 
- 6 x cameras, 

In [18]:
nusc.sensors

Every `sample_data` has a record on which `sensor` the data is collected from (note the "channel" key)

In [19]:
nusc.sample_datas[10]

### 10. `calibrated_sensor`

`calibrated_sensor` consists of the definition of a particular sensor (lidar/radar/camera) as calibrated on a particular vehicle. Let us look at an example.

In [20]:
nusc.calibrated_sensors[0]

Note that the `translation` and the `rotation` parameters are given with respect to the ego vehicle body frame. 

### 11. `ego_pose`

`ego_pose` contains information about the location (encoded in `translation`) and the orientation (encoded in `rotation`) of the ego vehicle, with respect to the global coordinate system.

In [21]:
nusc.ego_positions[0]

Note that the number of `ego_pose` records in our loaded database is the same as the number of `sample_data` records. These two records exhibit a one-to-one correspondence.

### 12. `log`

The `log` table contains log information from which the data was extracted. A `log` record corresponds to one journey of our ego vehicle along a predefined route. Let's check the number of logs and the metadata of a log.

In [22]:
print("Number of `logs` in our loaded database: {}".format(len(nusc.logs)))

Number of `logs` in our loaded database: 8


In [23]:
nusc.logs[0]

Notice that it contains a variety of information such as the date and location of the log collected. It also gives out information about the map from where the data was collected. Note that one log can contain multiple non-overlapping scenes.

### 13. `map`

Map information is stored as binary semantic masks from a top-down view. Let's check the number of maps and metadata of a map.

In [24]:
print("There are {} maps masks in the loaded dataset".format(len(nusc.maps)))

There are 4 maps masks in the loaded dataset


In [25]:
nusc.maps[0]

## nuScenes Basics

Let's get a bit technical.

The NuScenes class holds several tables. Each table is a list of records, and each record is a dictionary. For example the first record of the category table is stored at:

In [26]:
nusc.categories[0]

The category table is simple: it holds the fields `name` and `description`. It also has a `token` field, which is a unique record identifier. Since the record is a dictionary, the token can be accessed like so:

In [27]:
cat_token = nusc.categories[0].token
cat_token

'1fa93b757fc74fb197cdd60001ad8abf'

If you know the `token` for any record in the DB you can retrieve the record by doing

In [28]:
nusc.category(cat_token)

_As you can notice, we have recovered the same record!_

OK, that was easy. Let's try something harder. Let's look at the `sample_annotation` table.

In [29]:
nusc.annotations[0]

This also has a `token` field (they all do). In addition, it has several fields of the format [a-z]*\_token, _e.g._ instance_token. These are foreign keys in database terminology, meaning they point to another table. 
Using `nusc.get()` we can grab any of these in constant time. For example, let's look at the visibility record.

In [30]:
nusc.visibility(nusc.annotations[0].visibility_token)

The visibility records indicate how much of an object was visible when it was annotated.

Let's also grab the `instance_token`

In [31]:
one_instance = nusc.instance(nusc.annotations[0].instance_token)
one_instance

This points to the `instance` table. This table enumerate the object _instances_ we have encountered in each 
scene. This way we can connect all annotations of a particular object.

If you look carefully at the README tables, you will see that the sample_annotation table points to the instance table, 
but the instance table doesn't list all annotations that point to it. 

So how can we recover all sample_annotations for a particular object instance? There are two ways:

1. Directly use `instance.annotations`. Let's try it:

In [32]:
ann_tokens = one_instance.annotations
ann_tokens

This returns a list of all sample_annotation records with the `'instance_token'` == `one_instance['token']`. Let's store these in a set for now

In [33]:
ann_tokens_field2token = set(ann_tokens)

ann_tokens_field2token

{<_nuscenes.Annotation at 0x7fda40087af0>,
 <_nuscenes.Annotation at 0x7fda400893b0>}

The `nusc.field2token()` method is generic and can be used in any similar situation.

2. For certain situation, we provide some reverse indices in the tables themselves. This is one such example. 

The instance record has a field `first_annotation_token` which points to the first annotation in time of this instance. 
Recovering this record is easy.

In [34]:
ann_record = one_instance.annotations[0]
ann_record

Now we can traverse all annotations of this instance using the "next" field. Let's try it. 

In [35]:
ann_tokens_traverse = set()
ann_tokens_traverse.add(ann_record.token)
while not ann_record.next_token == "":
    ann_record = nusc.annotation(ann_record.next_token)
    ann_tokens_traverse.add(ann_record.token)

Finally, let's assert that we recovered the same ann_records as we did using nusc.field2token:

In [36]:
print(len(ann_tokens_field2token))
print(len(ann_tokens_traverse))
print(ann_tokens_traverse == ann_tokens_field2token)

39
39
False


## Reverse indexing and short-cuts

The nuScenes tables are normalized, meaning that each piece of information is only given once.
For example, there is one `map` record for each `log` record. Looking at the schema you will notice that the `map` table has a `log_token` field, but that the `log` table does not have a corresponding `map_token` field. But there are plenty of situations where you have a `log`, and want to find the corresponding `map`! So what to do? You can always use the `nusc.field2token()` method, but that is slow and inconvenient. We therefore add reverse mappings for some common situations including this one.

Further, there are situations where one needs to go through several tables to get a certain piece of information. 
Consider, for example, the category name (e.g. `human.pedestrian`) of a `sample_annotation`. The `sample_annotation` table doesn't hold this information since the category is an instance level constant. Instead the `sample_annotation` table points to a record in the `instance` table. This, in turn, points to a record in the `category` table, where finally the `name` fields stores the required information.

Since it is quite common to want to know the category name of an annotation, we add a `category_name` field to the `sample_annotation` table during initialization of the NuScenes class.

In this section, we list the short-cuts and reverse indices that are added to the `NuScenes` class during initialization. These are all created in the `NuScenes.__make_reverse_index__()` method.

### Reverse indices
We add two reverse indices by default.
* A `map_token` field is added to the `log` records.
* The `sample` records have shortcuts to all `sample_annotations` for that record as well as `sample_data` key-frames. Confer `nusc.list_sample()` method in the previous section for more details on this.

## Data Visualizations

We provide list and rendering methods. These are meant both as convenience methods during development, and as tutorials for building your own visualization methods. They are implemented in the NuScenesExplorer class, with shortcuts through the NuScenes class itself.

### List methods
There are three list methods available.

1. `list_categories()` lists all categories, counts and statistics of width/length/height in meters and aspect ratio.

In [37]:
nusc.categories

2. `list_attributes()` lists all attributes and counts.

In [38]:
nusc.attributes

3. `list_scenes()` lists all scenes in the loaded DB.

In [39]:
nusc.scenes